In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [ ]:
from proj1_helpers import *
from helpers2 import *

In [ ]:
#data_folder = Path("../data/")
DATA_TRAIN_PATH = "../data/train.csv"
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Data analysis

In [ ]:
print(y)
# -1 c'est pour 'b' et 1 c'est pour 's'
print(np.shape(y))
print(np.shape(tX))
# On a donc 30 features et 25'000 data différentes
print(tX.dtype)
print(y.dtype)

In [ ]:
plt.hist(y)
plt.show()
# Plus de -1 que de 1 --> donc plus de 'b' que de 'c'

In [ ]:
plt.figure(figsize=(20, 10))
plt.boxplot(tX[:,])
plt.show()
# Certains features ont pas mal d'outliers, peut etre enlever les données avec des outliers vraiment loin

In [ ]:
fig, axs = plt.subplots(5, 6, figsize=(15,15))

n = 0
for i in range(5) :
    for j in range(6) :
        axs[i,j].hist(tX[:,n])
        axs[i,j].set_title(n)
        n = n + 1
plt.show()

print(np.argmax(tX[:, 8]))

# Les features 0, 4, 5, 6, 12, 23, 24, 25, 26, 27, 28 semblent etre très bizarres

# 15, 18, 20 bizarres ? 0, 4, 5, 6, 23, 26 aussi 
# enlever?

In [ ]:
fig, axs = plt.subplots(5, 6, figsize=(15,15))

n = 0
for i in range(5) :
    for j in range(6) :
        axs[i,j].scatter(tX[:,n], y)
        axs[i,j].set_title(n)
        n = n + 1
plt.show()

#meme constat comment faire pour se debarrasser de ces valeurs ??

## Data filtering

### Outliers

In [ ]:
#regarder ou il y a des -999 ou bcp de outlier
sum = 0
nb_outliers = []
for col in range(tX.shape[1]) :
    sum = np.where(tX[:,col] == -999)[0].shape
    nb_outliers.append(sum)   
print(nb_outliers)
#Ceci confirme ce qu'on a vu dans les histogrammes

print(np.where(tX==-999)[0].shape)
#il y a 1580052 -999 dans les data
#peut etre que toutes les valeurs NA ont ete remplacées par une constante globale -999 ?  

### Mean and normalization

In [ ]:
# Changer par la valeur da la mean du feature en question sans les compter dedans
def filtering_with_mean(tX, index=0):
    if index == 0: 
        index = np.arange(tX.shape[1])
    tX_filtered = np.copy(tX)
    arr = []
    for ind in index :
        arr = np.delete(tX_filtered[:,ind], np.where(tX_filtered[:,ind]==-999))
        mean = np.mean(arr)
        tX_filtered[np.where(tX_filtered[:,ind]==-999), ind] = mean
    return tX_filtered

### Cutting features 

In [ ]:
def cut(tX, to_cut):
    cut_index = 100*np.ones(tX.shape[1])
    index_full = np.arange(tX.shape[1])
    for i in range(tX.shape[1]):
        for j in range(len(to_cut)):
            if index_full[i] == to_cut[j]:
                cut_index[i] = to_cut[j]
    index = index_full[~(index_full == cut_index)]
    index = index.reshape(-1)
    tX_cut = tX[:, index]
    return tX_cut

def keep(tX, to_keep):
    keep_index = 100*np.ones(tX.shape[1])
    index_full = np.arange(tX.shape[1])
    for i in range(tX.shape[1]):
        for j in range(len(to_keep)):
            if index_full[i] == to_keep[j]:
                keep_index[i] = to_keep[j]
    index = index_full[index_full == keep_index]
    index = index.reshape(-1)
    tX_kept = tX[:, index]
    return tX_kept

## Fonctions 

In [ ]:
from implementations import *
from cross_validation import *

#### Feature engineering

In [ ]:
tX_mean = filtering_with_mean(tX)
tX_norm, _, _ = standardize(tX)
tX_mean_norm, _, _ = standardize(tX_mean)

to_cut = [0, 4, 5, 6, 12, 23, 24, 25, 26, 27, 28]
tX_cut = cut(tX_mean_norm, to_cut)

to_keep = [13, 14, 17]
tX_kept = keep(tX_mean_norm, to_keep)

#### Choose x and generate test and train data 

In [ ]:
x1 = tX_mean
x2 = tX_kept

### Least squares

Cross validation to get parameters

In [97]:
degrees = np.arange(1,8)
degree_opt, lambda_opt = best_degree_selection(y, x1, degrees, k_fold=4, lambdas=0, fonction=0)

LinAlgError: Singular matrix

Result

In [ ]:
w_ls, loss_ls = least_squares(y, x1)
print("Least square loss {loss}".format(loss=loss_ls))

### Ridge regression 

Cross validation to find the oprimal lambda and degree

In [90]:
degree_opt, lambda_opt = best_degree_selection(y, x1, degrees=np.arange(1,8), k_fold=4, lambdas=np.logspace(-2, 2, 30))
print('CV done')

In [92]:
degree_opt_2, lambda_opt_2 = best_degree_selection(y, x1, degrees=np.arange(1,8), k_fold=4, lambdas=np.logspace(-4, 0, 30))
print('CV done')

CV done


Result

In [94]:
tX_tr = build_poly(x1, degree_opt)
w_rr, loss_rr = ridge_regression(y, x1, lambda_opt)
print("Ridge regression loss {loss} with optimal lambda {l} and degree {d}".format(loss=loss_rr, l=lambda_opt, d=degree_opt))

tX_tr = build_poly(x1, degree_opt_2)
w_rr, loss_rr = ridge_regression(y, x1, lambda_opt_2)
print("Ridge regression loss {loss} with optimal lambda {l} and degree {d}".format(loss=loss_rr, l=lambda_opt_2, d=degree_opt_2))

Ridge regression loss 0.348717909143505 with optimal lambda 0.06723357536499334 and degree 7
Ridge regression loss 0.34381954694688865 with optimal lambda 0.0004893900918477494 and degree 6


### Gradient descent

CV to find best gamma

In [64]:
# Si tu fais ca avec tous les features, ca prend beaucoup de temps (>1h) et ne converge pas
# en prenant x_kept on converge vers loss=0.42 et gamma_opt = 2.51 
# je pense qu'il faut trouver un just milieu entre 30 et 3 features 
max_iters = 50
k_fold = 4
initial_w = np.zeros(x.shape[1])
gammas = np.arange(0, 3, 0.03)
gamma_opt = cross_validation(y, x2, k_fold, gammas, fonction=2)

Gradient Descent(0/49): loss=0.5
Gradient Descent(1/49): loss=0.4634160843093332
Gradient Descent(2/49): loss=0.4542701053866668
Gradient Descent(3/49): loss=0.4519836106560001
Gradient Descent(4/49): loss=0.4514119869733334
Gradient Descent(5/49): loss=0.4512690810526668
Gradient Descent(6/49): loss=0.4512333545725
Gradient Descent(7/49): loss=0.4512244229524584
Gradient Descent(8/49): loss=0.4512221900474479
Gradient Descent(9/49): loss=0.45122163182119535
Gradient Descent(10/49): loss=0.45122149226463204
Gradient Descent(11/49): loss=0.4512214573754914
Gradient Descent(12/49): loss=0.4512214486532063
Gradient Descent(13/49): loss=0.4512214464726348
Gradient Descent(14/49): loss=0.45122144592749225
Gradient Descent(15/49): loss=0.45122144579120627
Gradient Descent(16/49): loss=0.4512214457571348
Gradient Descent(17/49): loss=0.45122144574861717
Gradient Descent(18/49): loss=0.45122144574648737
Gradient Descent(19/49): loss=0.45122144574595524
Gradient Descent(20/49): loss=0.451221445

Gradient Descent(18/49): loss=0.45017306880007824
Gradient Descent(19/49): loss=0.4501730688000174
Gradient Descent(20/49): loss=0.4501730688000039
Gradient Descent(21/49): loss=0.4501730688000008
Gradient Descent(22/49): loss=0.4501730687999999
Gradient Descent(23/49): loss=0.4501730688
Gradient Descent(24/49): loss=0.4501730688
Gradient Descent(25/49): loss=0.4501730688
Gradient Descent(26/49): loss=0.4501730688000001
Gradient Descent(27/49): loss=0.4501730688
Gradient Descent(28/49): loss=0.45017306879999985
Gradient Descent(29/49): loss=0.4501730688000001
Gradient Descent(30/49): loss=0.45017306880000024
Gradient Descent(31/49): loss=0.4501730687999999
Gradient Descent(32/49): loss=0.4501730688000001
Gradient Descent(33/49): loss=0.45017306879999985
Gradient Descent(34/49): loss=0.4501730687999999
Gradient Descent(35/49): loss=0.4501730688000001
Gradient Descent(36/49): loss=0.4501730687999999
Gradient Descent(37/49): loss=0.45017306879999985
Gradient Descent(38/49): loss=0.4501730

Gradient Descent(24/49): loss=0.45017306879999985
Gradient Descent(25/49): loss=0.4501730688
Gradient Descent(26/49): loss=0.4501730688000001
Gradient Descent(27/49): loss=0.4501730688
Gradient Descent(28/49): loss=0.4501730687999999
Gradient Descent(29/49): loss=0.4501730687999999
Gradient Descent(30/49): loss=0.45017306879999985
Gradient Descent(31/49): loss=0.45017306879999985
Gradient Descent(32/49): loss=0.4501730688000001
Gradient Descent(33/49): loss=0.4501730688000001
Gradient Descent(34/49): loss=0.45017306879999985
Gradient Descent(35/49): loss=0.4501730688000001
Gradient Descent(36/49): loss=0.4501730688000002
Gradient Descent(37/49): loss=0.4501730688000001
Gradient Descent(38/49): loss=0.4501730688000001
Gradient Descent(39/49): loss=0.4501730688
Gradient Descent(40/49): loss=0.4501730688000002
Gradient Descent(41/49): loss=0.4501730688
Gradient Descent(42/49): loss=0.4501730688
Gradient Descent(43/49): loss=0.4501730688000002
Gradient Descent(44/49): loss=0.4501730688
Gra

Gradient Descent(45/49): loss=0.4505763199999999
Gradient Descent(46/49): loss=0.4505763199999999
Gradient Descent(47/49): loss=0.4505763199999999
Gradient Descent(48/49): loss=0.4505763199999999
Gradient Descent(49/49): loss=0.4505763199999999
Gradient Descent(0/49): loss=0.5
Gradient Descent(1/49): loss=0.45854897593472016
Gradient Descent(2/49): loss=0.45158105878934623
Gradient Descent(3/49): loss=0.4504097519172092
Gradient Descent(4/49): loss=0.45021285523200283
Gradient Descent(5/49): loss=0.4501797568992198
Gradient Descent(6/49): loss=0.45017419306947903
Gradient Descent(7/49): loss=0.45017325778969935
Gradient Descent(8/49): loss=0.4501731005691686
Gradient Descent(9/49): loss=0.4501730741403972
Gradient Descent(10/49): loss=0.45017306969772086
Gradient Descent(11/49): loss=0.4501730689509068
Gradient Descent(12/49): loss=0.4501730688253676
Gradient Descent(13/49): loss=0.4501730688042643
Gradient Descent(14/49): loss=0.4501730688007167
Gradient Descent(15/49): loss=0.4501730

Gradient Descent(17/49): loss=0.4512214457457778
Gradient Descent(18/49): loss=0.4512214457457777
Gradient Descent(19/49): loss=0.4512214457457777
Gradient Descent(20/49): loss=0.4512214457457779
Gradient Descent(21/49): loss=0.4512214457457777
Gradient Descent(22/49): loss=0.4512214457457778
Gradient Descent(23/49): loss=0.4512214457457779
Gradient Descent(24/49): loss=0.4512214457457777
Gradient Descent(25/49): loss=0.4512214457457777
Gradient Descent(26/49): loss=0.4512214457457778
Gradient Descent(27/49): loss=0.4512214457457777
Gradient Descent(28/49): loss=0.4512214457457778
Gradient Descent(29/49): loss=0.4512214457457779
Gradient Descent(30/49): loss=0.4512214457457779
Gradient Descent(31/49): loss=0.4512214457457777
Gradient Descent(32/49): loss=0.4512214457457778
Gradient Descent(33/49): loss=0.4512214457457778
Gradient Descent(34/49): loss=0.4512214457457778
Gradient Descent(35/49): loss=0.4512214457457778
Gradient Descent(36/49): loss=0.4512214457457777
Gradient Descent(37/

...

Gradient Descent(34/49): loss=1.5229771401456106e+17
Gradient Descent(35/49): loss=5.325698761374756e+17
Gradient Descent(36/49): loss=1.8623435998652616e+18
Gradient Descent(37/49): loss=6.51242933436912e+18
Gradient Descent(38/49): loss=2.27733141393548e+19
Gradient Descent(39/49): loss=7.963600221391605e+19
Gradient Descent(40/49): loss=2.784791361418288e+20
Gradient Descent(41/49): loss=9.738136911743584e+20
Gradient Descent(42/49): loss=3.4053290966677533e+21
Gradient Descent(43/49): loss=1.1908095318137388e+22
Gradient Descent(44/49): loss=4.164141851799072e+22
Gradient Descent(45/49): loss=1.4561587641556028e+23
Gradient Descent(46/49): loss=5.092041582376027e+23
Gradient Descent(47/49): loss=1.780636020941152e+24
Gradient Descent(48/49): loss=6.226706101629432e+24
Gradient Descent(49/49): loss=2.177416856678929e+25
Gradient Descent(0/49): loss=0.5
Gradient Descent(1/49): loss=0.6273120266035203
Gradient Descent(2/49): loss=1.0869084426421898
Gradient Descent(3/49): loss=2.74605

Gradient Descent(2/49): loss=1.0995226188915703
Gradient Descent(3/49): loss=2.794324944630535
Gradient Descent(4/49): loss=8.912561340547914
Gradient Descent(5/49): loss=30.999394729808962
Gradient Descent(6/49): loss=110.73286326504692
Gradient Descent(7/49): loss=398.5706846772683
Gradient Descent(8/49): loss=1437.6652199753926
Gradient Descent(9/49): loss=5188.79649240193
Gradient Descent(10/49): loss=18730.38038586173
Gradient Descent(11/49): loss=67615.49824125464
Gradient Descent(12/49): loss=244090.77369923596
Gradient Descent(13/49): loss=881166.5181026418
Gradient Descent(14/49): loss=3181009.9553990187
Gradient Descent(15/49): loss=11483444.76403962
Gradient Descent(16/49): loss=41455234.42323413
Gradient Descent(17/49): loss=149653395.09291255
Gradient Descent(18/49): loss=540248755.1104509
Gradient Descent(19/49): loss=1950298004.773686
Gradient Descent(20/49): loss=7040575796.05764
Gradient Descent(21/49): loss=25416478622.59136
Gradient Descent(22/49): loss=91753487826.3

Gradient Descent(37/49): loss=6.686147384758718e+19
Gradient Descent(38/49): loss=2.4905230393489677e+20
Gradient Descent(39/49): loss=9.276949269270158e+20
Gradient Descent(40/49): loss=3.455570833310846e+21
Gradient Descent(41/49): loss=1.287165579699927e+22
Gradient Descent(42/49): loss=4.794563067824506e+22
Gradient Descent(43/49): loss=1.7859267971341116e+23
Gradient Descent(44/49): loss=6.652398726644485e+23
Gradient Descent(45/49): loss=2.477952001687559e+24
Gradient Descent(46/49): loss=9.230123411086321e+24
Gradient Descent(47/49): loss=3.4381286693957358e+25
Gradient Descent(48/49): loss=1.2806685480632027e+26
Gradient Descent(49/49): loss=4.7703622746802827e+26
Gradient Descent(0/49): loss=0.5
Gradient Descent(1/49): loss=0.6357734048268806
Gradient Descent(2/49): loss=1.1415157604664925
Gradient Descent(3/49): loss=3.0253554609887257
Gradient Descent(4/49): loss=10.042469961463254
Gradient Descent(5/49): loss=36.18051976427863
Gradient Descent(6/49): loss=133.54214147479433

Gradient Descent(6/49): loss=159.30815376955013
Gradient Descent(7/49): loss=610.7178458501925
Gradient Descent(8/49): loss=2344.85331894718
Gradient Descent(9/49): loss=9006.708152397143
Gradient Descent(10/49): loss=34598.88968057494
Gradient Descent(11/49): loss=132913.81423923324
Gradient Descent(12/49): loss=510600.42842374684
Gradient Descent(13/49): loss=1961521.3254749414
Gradient Descent(14/49): loss=7535379.04358685
Gradient Descent(15/49): loss=28947910.85348521
Gradient Descent(16/49): loss=111206293.05438988
Gradient Descent(17/49): loss=427210094.117368
Gradient Descent(18/49): loss=1641170296.2808027
Gradient Descent(19/49): loss=6304719808.911851
Gradient Descent(20/49): loss=24220211616.636677
Gradient Descent(21/49): loss=93044364945.19864
Gradient Descent(22/49): loss=357439232372.17487
Gradient Descent(23/49): loss=1373138555079.698
Gradient Descent(24/49): loss=5275049073192.818
Gradient Descent(25/49): loss=20264628519577.66
Gradient Descent(26/49): loss=778485969

Gradient Descent(41/49): loss=1.6030284269017955e+23
Gradient Descent(42/49): loss=6.348152873374325e+23
Gradient Descent(43/49): loss=2.5139320193850513e+24
Gradient Descent(44/49): loss=9.955422189967363e+24
Gradient Descent(45/49): loss=3.942446741449185e+25
Gradient Descent(46/49): loss=1.561248334081288e+26
Gradient Descent(47/49): loss=6.182699527795543e+26
Gradient Descent(48/49): loss=2.4484108400022774e+27
Gradient Descent(49/49): loss=9.695951767493764e+27
Gradient Descent(0/49): loss=0.5
Gradient Descent(1/49): loss=0.6462990351680001
Gradient Descent(2/49): loss=1.225657844336867
Gradient Descent(3/49): loss=3.519976664526195
Gradient Descent(4/49): loss=12.605708624359469
Gradient Descent(5/49): loss=48.58611575849568
Gradient Descent(6/49): loss=191.07212605037736
Gradient Descent(7/49): loss=755.3309754072304
Gradient Descent(8/49): loss=2989.8524447454747
Gradient Descent(9/49): loss=11838.780915472218
Gradient Descent(10/49): loss=46881.422552393684
Gradient Descent(11

result

In [65]:
w_gd, loss_gd = least_squares_GD(y, x2, gamma_opt, max_iters=max_iters)
print("Gradient descent regression loss {loss} with optimal gamma {g}".format(loss=loss_gd, g=gamma_opt))

1.8200000000000012
Gradient Descent(0/49): loss=0.5
Gradient Descent(1/49): loss=0.44440890578566167
Gradient Descent(2/49): loss=0.44024983391625105
Gradient Descent(3/49): loss=0.4376256471028858
Gradient Descent(4/49): loss=0.4354831764450516
Gradient Descent(5/49): loss=0.4337218497498242
Gradient Descent(6/49): loss=0.43227362758912535
Gradient Descent(7/49): loss=0.4310828460324516
Gradient Descent(8/49): loss=0.4301037415182902
Gradient Descent(9/49): loss=0.4292986856635453
Gradient Descent(10/49): loss=0.4286367390275766
Gradient Descent(11/49): loss=0.42809246206268226
Gradient Descent(12/49): loss=0.427644937435685
Gradient Descent(13/49): loss=0.4272769661447635
Gradient Descent(14/49): loss=0.42697440653710034
Gradient Descent(15/49): loss=0.42672563082484527
Gradient Descent(16/49): loss=0.42652107821252283
Gradient Descent(17/49): loss=0.42635288746192346
Gradient Descent(18/49): loss=0.4262145947734255
Gradient Descent(19/49): loss=0.4261008853728879
Gradient Descent(20

### Stochastic gradient descent

cv for gamma

In [70]:
max_iters = 50
k_fold = 4
initial_w = np.zeros(x.shape[1])
gammas = np.arange(0, 3, 0.03)
gamma_opt = cross_validation(y, x2, k_fold, gammas, fonction=2)

Gradient Descent(0/49): loss=0.5
Gradient Descent(1/49): loss=0.5
Gradient Descent(2/49): loss=0.5
Gradient Descent(3/49): loss=0.5
Gradient Descent(4/49): loss=0.5
Gradient Descent(5/49): loss=0.5
Gradient Descent(6/49): loss=0.5
Gradient Descent(7/49): loss=0.5
Gradient Descent(8/49): loss=0.5
Gradient Descent(9/49): loss=0.5
Gradient Descent(10/49): loss=0.5
Gradient Descent(11/49): loss=0.5
Gradient Descent(12/49): loss=0.5
Gradient Descent(13/49): loss=0.5
Gradient Descent(14/49): loss=0.5
Gradient Descent(15/49): loss=0.5
Gradient Descent(16/49): loss=0.5
Gradient Descent(17/49): loss=0.5
Gradient Descent(18/49): loss=0.5
Gradient Descent(19/49): loss=0.5
Gradient Descent(20/49): loss=0.5
Gradient Descent(21/49): loss=0.5
Gradient Descent(22/49): loss=0.5
Gradient Descent(23/49): loss=0.5
Gradient Descent(24/49): loss=0.5
Gradient Descent(25/49): loss=0.5
Gradient Descent(26/49): loss=0.5
Gradient Descent(27/49): loss=0.5
Gradient Descent(28/49): loss=0.5
Gradient Descent(29/49):

Gradient Descent(0/49): loss=0.5
Gradient Descent(1/49): loss=0.49704487103192735
Gradient Descent(2/49): loss=0.4942643901858682
Gradient Descent(3/49): loss=0.49164823575781075
Gradient Descent(4/49): loss=0.48918669605645165
Gradient Descent(5/49): loss=0.4868706333514427
Gradient Descent(6/49): loss=0.4846914499523
Gradient Descent(7/49): loss=0.48264105629204657
Gradient Descent(8/49): loss=0.4807118408971141
Gradient Descent(9/49): loss=0.4788966421320223
Gradient Descent(10/49): loss=0.4771887216139473
Gradient Descent(11/49): loss=0.47558173919849034
Gradient Descent(12/49): loss=0.4740697294437869
Gradient Descent(13/49): loss=0.47264707946558676
Gradient Descent(14/49): loss=0.47130850810109803
Gradient Descent(15/49): loss=0.4700490463042507
Gradient Descent(16/49): loss=0.46886401869959693
Gradient Descent(17/49): loss=0.4677490262263778
Gradient Descent(18/49): loss=0.46669992980832653
Gradient Descent(19/49): loss=0.4657128349885821
Gradient Descent(20/49): loss=0.4647840

Gradient Descent(0/49): loss=0.5
Gradient Descent(1/49): loss=0.4941797459918164
Gradient Descent(2/49): loss=0.48903696955018555
Gradient Descent(3/49): loss=0.4844928122863606
Gradient Descent(4/49): loss=0.48047759492804476
Gradient Descent(5/49): loss=0.4769297488702371
Gradient Descent(6/49): loss=0.47379487209355775
Gradient Descent(7/49): loss=0.4710248949736843
Gradient Descent(8/49): loss=0.468577343190564
Gradient Descent(9/49): loss=0.46641468643499895
Gradient Descent(10/49): loss=0.46450376292578155
Gradient Descent(11/49): loss=0.46281527091303715
Gradient Descent(12/49): loss=0.4613233193705762
Gradient Descent(13/49): loss=0.4600050309876576
Gradient Descent(14/49): loss=0.4588401913725109
Gradient Descent(15/49): loss=0.4578109390885672
Gradient Descent(16/49): loss=0.4569014917704742
Gradient Descent(17/49): loss=0.4560979041202077
Gradient Descent(18/49): loss=0.45538785407243226
Gradient Descent(19/49): loss=0.45476045385021757
Gradient Descent(20/49): loss=0.454206

Gradient Descent(9/49): loss=0.4591544131042345
Gradient Descent(10/49): loss=0.4575803943712837
Gradient Descent(11/49): loss=0.45627694945852737
Gradient Descent(12/49): loss=0.45519756672627376
Gradient Descent(13/49): loss=0.4543037298856945
Gradient Descent(14/49): loss=0.4535635435980106
Gradient Descent(15/49): loss=0.4529505953331798
Gradient Descent(16/49): loss=0.4524430128750734
Gradient Descent(17/49): loss=0.4520226838415157
Gradient Descent(18/49): loss=0.45167460936882625
Gradient Descent(19/49): loss=0.4513863688979921
Gradient Descent(20/49): loss=0.45114767696409447
Gradient Descent(21/49): loss=0.4509500161736339
Gradient Descent(22/49): loss=0.45078633327305323
Gradient Descent(23/49): loss=0.4506507874630826
Gradient Descent(24/49): loss=0.450538541977846
Gradient Descent(25/49): loss=0.45044559149152164
Gradient Descent(26/49): loss=0.45036861919379617
Gradient Descent(27/49): loss=0.4503048784340498
Gradient Descent(28/49): loss=0.45025209471090394
Gradient Desce

Gradient Descent(32/49): loss=0.45123509534631606
Gradient Descent(33/49): loss=0.45123201599643464
Gradient Descent(34/49): loss=0.4512296313478865
Gradient Descent(35/49): loss=0.4512277846760506
Gradient Descent(36/49): loss=0.45122635461338123
Gradient Descent(37/49): loss=0.45122524717284995
Gradient Descent(38/49): loss=0.4512243895709025
Gradient Descent(39/49): loss=0.45122372544395434
Gradient Descent(40/49): loss=0.4512232111440456
Gradient Descent(41/49): loss=0.4512228128701963
Gradient Descent(42/49): loss=0.45122250444692763
Gradient Descent(43/49): loss=0.4512222656039483
Gradient Descent(44/49): loss=0.451222080643945
Gradient Descent(45/49): loss=0.45122193741091843
Gradient Descent(46/49): loss=0.45122182649126275
Gradient Descent(47/49): loss=0.45122174059508136
Gradient Descent(48/49): loss=0.45122167407707825
Gradient Descent(49/49): loss=0.45122162256553705
Gradient Descent(0/49): loss=0.5
Gradient Descent(1/49): loss=0.48871950769547934
Gradient Descent(2/49): lo

...

Gradient Descent(27/49): loss=5396148283024.358
Gradient Descent(28/49): loss=17874201572689.887
Gradient Descent(29/49): loss=59206505289373.52
Gradient Descent(30/49): loss=196115628120511.84
Gradient Descent(31/49): loss=649613406586437.1
Gradient Descent(32/49): loss=2151779447976736.0
Gradient Descent(33/49): loss=7127554243478860.0
Gradient Descent(34/49): loss=2.3609310676101364e+16
Gradient Descent(35/49): loss=7.820348068351845e+16
Gradient Descent(36/49): loss=2.5904120941608173e+17
Gradient Descent(37/49): loss=8.580481020698701e+17
Gradient Descent(38/49): loss=2.8421985332964465e+18
Gradient Descent(39/49): loss=9.41449842169085e+18
Gradient Descent(40/49): loss=3.1184584572006273e+19
Gradient Descent(41/49): loss=1.032958179363178e+20
Gradient Descent(42/49): loss=3.421570673322757e+20
Gradient Descent(43/49): loss=1.1333610698313309e+21
Gradient Descent(44/49): loss=3.7541452077094493e+21
Gradient Descent(45/49): loss=1.243523058601694e+22
Gradient Descent(46/49): loss=4

Gradient Descent(0/49): loss=0.5
Gradient Descent(1/49): loss=0.6197288648
Gradient Descent(2/49): loss=1.0295009045779895
Gradient Descent(3/49): loss=2.4319457107180638
Gradient Descent(4/49): loss=7.231813059732662
Gradient Descent(5/49): loss=23.65935906173643
Gradient Descent(6/49): loss=79.882635253597
Gradient Descent(7/49): loss=272.3067980202014
Gradient Descent(8/49): loss=930.878495088877
Gradient Descent(9/49): loss=3184.840128306696
Gradient Descent(10/49): loss=10899.023817993857
Gradient Descent(11/49): loss=37300.817495950636
Gradient Descent(12/49): loss=127660.95635874868
Gradient Descent(13/49): loss=436918.5316166884
Gradient Descent(14/49): loss=1495352.5829371074
Gradient Descent(15/49): loss=5117843.123580833
Gradient Descent(16/49): loss=17515816.998934437
Gradient Descent(17/49): loss=59947882.587333374
Gradient Descent(18/49): loss=205171627.06364113
Gradient Descent(19/49): loss=702199892.533807
Gradient Descent(20/49): loss=2403279131.105337
Gradient Descent

Gradient Descent(24/49): loss=719516743795.2367
Gradient Descent(25/49): loss=2543059979268.8213
Gradient Descent(26/49): loss=8988191190727.098
Gradient Descent(27/49): loss=31767862944505.95
Gradient Descent(28/49): loss=112280334791052.02
Gradient Descent(29/49): loss=396843615285501.06
Gradient Descent(30/49): loss=1402604073865095.2
Gradient Descent(31/49): loss=4957363838668897.0
Gradient Descent(32/49): loss=1.7521306751392288e+16
Gradient Descent(33/49): loss=6.192730658212122e+16
Gradient Descent(34/49): loss=2.188758723838247e+17
Gradient Descent(35/49): loss=7.735948833534373e+17
Gradient Descent(36/49): loss=2.734193755724548e+18
Gradient Descent(37/49): loss=9.663734410232318e+18
Gradient Descent(38/49): loss=3.415550289952748e+19
Gradient Descent(39/49): loss=1.2071920944809966e+20
Gradient Descent(40/49): loss=4.266699738733706e+20
Gradient Descent(41/49): loss=1.5080223556579416e+21
Gradient Descent(42/49): loss=5.329954213837418e+21
Gradient Descent(43/49): loss=1.8838

Gradient Descent(33/49): loss=4.821066250276466e+17
Gradient Descent(34/49): loss=1.8144564939538954e+18
Gradient Descent(35/49): loss=6.828888460644255e+18
Gradient Descent(36/49): loss=2.570120461048008e+19
Gradient Descent(37/49): loss=9.672905367200729e+19
Gradient Descent(38/49): loss=3.640494663999355e+20
Gradient Descent(39/49): loss=1.3701365717428868e+21
Gradient Descent(40/49): loss=5.156646001411695e+21
Gradient Descent(41/49): loss=1.9407552890913914e+22
Gradient Descent(42/49): loss=7.30422660602483e+22
Gradient Descent(43/49): loss=2.7490187254432858e+23
Gradient Descent(44/49): loss=1.0346206875077859e+24
Gradient Descent(45/49): loss=3.893898419503885e+24
Gradient Descent(46/49): loss=1.4655076091646254e+25
Gradient Descent(47/49): loss=5.5155844378525495e+25
Gradient Descent(48/49): loss=2.075845359030059e+26
Gradient Descent(49/49): loss=7.812651593245695e+26
Gradient Descent(0/49): loss=0.5
Gradient Descent(1/49): loss=0.6381860307303765
Gradient Descent(2/49): loss=

Gradient Descent(35/49): loss=2.0493908345583387e+19
Gradient Descent(36/49): loss=7.953480889837597e+19
Gradient Descent(37/49): loss=3.086666398537024e+20
Gradient Descent(38/49): loss=1.197904362608188e+21
Gradient Descent(39/49): loss=4.6489470408457925e+21
Gradient Descent(40/49): loss=1.8042098570817712e+22
Gradient Descent(41/49): loss=7.00195803434803e+22
Gradient Descent(42/49): loss=2.7173898935498213e+23
Gradient Descent(43/49): loss=1.0545918437878621e+24
Gradient Descent(44/49): loss=4.092765486556138e+24
Gradient Descent(45/49): loss=1.588361357677405e+25
Gradient Descent(46/49): loss=6.164271593010941e+25
Gradient Descent(47/49): loss=2.3922921625313892e+26
Gradient Descent(48/49): loss=9.284246653568e+26
Gradient Descent(49/49): loss=3.6031232837832777e+27
Gradient Descent(0/49): loss=0.5
Gradient Descent(1/49): loss=0.642384679712
Gradient Descent(2/49): loss=1.1949653832062601
Gradient Descent(3/49): loss=3.3394758353971814
Gradient Descent(4/49): loss=11.662106449304

result

In [71]:
w_sgd, loss_sgd = least_squares_SGD(y, x2, gamma_opt, max_iters=50)
print("Stochastic gradient descent regression loss {loss} with optimal gamma {g}".format(loss=loss_gd, g=gamma_opt))

SGD(0/49): loss=0.5, w0=-0.037348050780443134, w1=-0.0892131248343261
SGD(1/49): loss=0.5334786296601186, w0=-0.03688527826399847, w1=-0.1683343714900538
SGD(2/49): loss=0.5666277427808751, w0=0.016073308438578675, w1=-0.06889369565858076
SGD(3/49): loss=0.5205240833028542, w0=0.03710200199100139, w1=0.02180359228194688
SGD(4/49): loss=0.48966832006871264, w0=0.06427849231935809, w1=0.10473294434706487
SGD(5/49): loss=0.4679893792880534, w0=0.07211117833667495, w1=0.17941902723405947
SGD(6/49): loss=0.45544922869280396, w0=0.10602379689792471, w1=0.24704105822688263
SGD(7/49): loss=0.44727542482689847, w0=0.1413191089654544, w1=0.3113069704501027
SGD(8/49): loss=0.44383260689406634, w0=0.12651207208113555, w1=0.19607840354468647
SGD(9/49): loss=0.4503922329765159, w0=0.16456291735283687, w1=0.26390987755500905
SGD(10/49): loss=0.4436091353985919, w0=0.11211471266205315, w1=0.15353581512444991
SGD(11/49): loss=0.4566045380180913, w0=0.14204070343934774, w1=0.22608976004896775
SGD(12/49)

## Generate predictions and save ouput in csv format for submission:

In [95]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

tX_test1 = filtering_with_mean(tX_test)

tX_test2 = keep(tX_test, to_keep)
tX_test2 = standardize(filtering_with_mean(tX_test))

In [ ]:
OUTPUT_PATH = '../data/pred_ls.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w_ls, tX_test1)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [ ]:
OUTPUT_PATH = '../data/pred_rr.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w_rr, tX_test1)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [96]:
OUTPUT_PATH = '../data/pred_sg.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w_gd, tX_test2)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

ValueError: setting an array element with a sequence.

In [ ]:
OUTPUT_PATH = '../data/pred_sgd.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w_sgd, tX_test2)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)